In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv('../_data/tb-rsqa-cleaned.csv')

In [11]:
df.columns

Index(['province', 'district', 'hf_tb_type', 'hf_name', 'hf_type',
       'genexpert_site', 'fy', 'summary_group', 'no', 'normes', 'item_name',
       'num', 'den'],
      dtype='object')

In [12]:
df.head()

,province,district,hf_tb_type,hf_name,hf_type,genexpert_site,fy,summary_group,no,normes,item_name,num,den
0,North,Gakenke,CT,ruli cs,health center,NO,2020-2021,"Are algorithms for TB screening, TB diagnosis ...",1,Toute FOSA doit rendre disponible des algorith...,"Les services VIH/ARV, Consultations externes, ...",3.0,4.0
1,North,Gakenke,CT,ruli cs,health center,NO,2020-2021,"Are TB monitoring tools existing, update and w...",2,Les prestataires doivent avoir les connaissanc...,"Les prestataires des services ARV, Consultatio...",4.0,4.0
2,North,Gakenke,CT,ruli cs,health center,NO,2020-2021,Is active case finding conducted in TB high ri...,3,NaN,Sur 10 nouveaux prisonniers entrés au cours de...,NaN,10.0
3,North,Gakenke,CT,ruli cs,health center,NO,2020-2021,Is active case finding conducted in TB high ri...,4,NaN,Sur 10 nouveaux prisonniers sortants au cours ...,NaN,10.0
4,North,Gakenke,CT,ruli cs,health center,NO,2020-2021,Are TB diagnostics continuously functional (mi...,5,NaN,Sur les 10 derniers patients hospitalises( 5 p...,NaN,10.0


`Data cleaning`

In [13]:
df['fy'].unique()

array(['2020-2021', '2021-2022', '2022-2023', '2023-2024'], dtype=object)

In [14]:
df['fy_year'] = df['fy'].str.extract(r'(\d{4})').astype(int)
df['fy_year'].unique()

array([2020, 2021, 2022, 2023])

In [15]:
df.columns

Index(['province', 'district', 'hf_tb_type', 'hf_name', 'hf_type',
       'genexpert_site', 'fy', 'summary_group', 'no', 'normes', 'item_name',
       'num', 'den', 'fy_year'],
      dtype='object')

In [16]:
summary_item_scores = (
    df.groupby(
        ['summary_group', "genexpert_or_microscopy", 'item_name', 'hf_name', 'hf_tb_type', 'hf_type', 'fy']
    )
    .agg({'num': 'sum', 'den': 'sum'})
    .reset_index()
)

KeyError: 'genexpert_or_microscopy'

`screening, diagnosis, and treatment`

In [ ]:
# Define mapping of summary_group to section
section_mapping = {
    "Is active case finding conducted in TB high risk groups?[Questions 2, 8 and 9]": "screening",
    "Are TB diagnostics continuously functional (microscopy and Expert) and their results available timely (microscopy, expert and culture)?[Questions 3, 4, 5, 6 and 7]": "diagnosis",
    "Are TB patients early initiated on TB treatment and on ART (if indicated), and their bacteriological control performed according to guidelines?[Questions 16, 17, 18 and 19]": "treatment",
    "Does the BMI monitored for TB Patients and nutritional support provided to the eligible patients?[Questions New_70 and New_71]": "treatment"
}

summary_item_scores["section"] = summary_item_scores["summary_group"].map(section_mapping)

`average score`

In [ ]:
# Compute the average score as (sum of num / sum of den) * 100
summary_item_scores['avg_score'] = (summary_item_scores['num'] / summary_item_scores['den']) * 100

`Filter only rows which are either 'screening', 'treatment', or 'diagnostic'`

In [ ]:
filtered_summary = summary_item_scores[summary_item_scores["section"].isin(["screening", "treatment", "diagnosis"])]

`remove prisoners questions`

In [ ]:
# Define the items to exclude
items_to_exclude = [
    "Sur 10 nouveaux prisonniers entrÃ©s au cours de la pÃ©riode evaluÃ©e, combien ont beneficiÃ© du screening TB Ã¡ l'entrÃ©e",
    "Sur 10 nouveaux prisonniers sortants au cours de la pÃ©riode evaluÃ©e, combien ont beneficiÃ© du screening TB Ã¡ la sortie"
]

# Filter out rows where item_name is in items_to_exclude
filtered_summary = filtered_summary[~filtered_summary["item_name"].isin(items_to_exclude)]

In [ ]:
filtered_summary.columns

Index(['summary_group', 'genexpert_or_microscopy', 'item_name', 'hf_name',
       'hf_tb_type', 'hf_type', 'fy', 'num', 'den', 'section', 'avg_score'],
      dtype='object')

In [ ]:
filtered_summary.shape

(6700, 11)

In [ ]:
# Define valid cases
valid_df = filtered_summary[
    (filtered_summary["num"] <= filtered_summary["den"]) &  # num must not be greater than den
    ~(filtered_summary["num"].isna() & filtered_summary["den"] > 0) &  # Exclude cases where num is NaN but den > 0
    ~((filtered_summary["num"] == 0) & (filtered_summary["den"] == 0)) &  # Exclude cases where both num and den are 0
    ~(filtered_summary["den"].isna())  # Exclude cases where den is NaN (new fix)
]

invalid_df = filtered_summary[
    (filtered_summary["num"] > filtered_summary["den"]) |  # Invalid if num > den
    ((filtered_summary["num"].isna()) & (filtered_summary["den"] > 0)) |  # Invalid if num is NaN but den > 0
    ((filtered_summary["num"] == 0) & (filtered_summary["den"] == 0)) |  # Invalid if both num and den are 0
    (filtered_summary["den"].isna()) |  # Invalid if den is NaN
    ((filtered_summary["num"].isna()) & (filtered_summary["den"] == 0))  # NEW FIX: Invalid if num is NaN and den = 0
]



In [ ]:
print("valid size: ", valid_df.shape)
print("invalid size: ", invalid_df.shape)

In [ ]:
stop

In [ ]:
valid_df.columns

Index(['summary_group', 'genexpert_or_microscopy', 'item_name', 'hf_name',
       'hf_tb_type', 'hf_type', 'fy', 'num', 'den', 'section', 'avg_score'],
      dtype='object')

In [ ]:
fy_summary = valid_df.groupby(
    ['section', 'hf_tb_type', 'hf_type', 'fy']
).apply(lambda x: (x["num"].sum() / x["den"].sum()) * 100).reset_index(name="overall_score")
fy_summary

,section,hf_tb_type,hf_type,fy,overall_score
0,diagnosis,CDT,health center,2020-2021,80.508965
1,diagnosis,CDT,health center,2021-2022,43.314567
2,diagnosis,CDT,health center,2022-2023,41.993958
3,diagnosis,CDT,health center,2023-2024,54.724409
4,diagnosis,CDT,"hospital (dh,ph,rh)",2020-2021,83.333333
5,diagnosis,CDT,"hospital (dh,ph,rh)",2021-2022,80.077745
6,diagnosis,CDT,"hospital (dh,ph,rh)",2022-2023,79.719388
7,diagnosis,CDT,"hospital (dh,ph,rh)",2023-2024,82.928724
8,diagnosis,CT,health center,2020-2021,40.702341
9,diagnosis,CT,health center,2021-2022,32.321429


In [ ]:
# hf_counts = fy_summary.groupby(["section", "hf_tb_type", "fy", "hf_type"])["hf_name"].nunique().reset_index(name="hf_count")
# overall_scores = fy_summary.groupby(["section", "hf_tb_type", "fy", "hf_type"]).apply(lambda x: (x["percentage"].sum() / x["percentage"].shape[0])).reset_index(name="overall_score")
# invalid_result_df = pd.merge(hf_counts, overall_scores, on=["section", "fy", "hf_type"], how="left")
# invalid_result_df

In [ ]:
# First calculate the overall_score as you were doing
fy_summary = valid_df.groupby(["genexpert_or_microscopy", "hf_tb_type", 'section', 'hf_type', 'fy']).apply(
    lambda x: (x["num"].sum() / x["den"].sum()) * 100
).reset_index(name="overall_score")

# Now calculate the hf_count by counting unique health facilities in each group
hf_counts = valid_df.groupby(["genexpert_or_microscopy", "hf_tb_type", 'section', 'hf_type', 'fy'])["hf_name"].nunique().reset_index(name="hf_count")

# Merge the two dataframes to get all the columns you need
result_df = pd.merge(fy_summary, hf_counts, on=["genexpert_or_microscopy", "hf_tb_type", 'section', 'hf_type', 'fy'], how="left")

In [ ]:
result_df

,genexpert_or_microscopy,hf_tb_type,section,hf_type,fy,overall_score,hf_count
0,no,CDT,diagnosis,health center,2020-2021,80.058867,23
1,no,CDT,diagnosis,health center,2021-2022,40.008503,24
2,no,CDT,diagnosis,health center,2022-2023,41.941748,10
3,no,CDT,diagnosis,health center,2023-2024,48.507463,21
4,no,CDT,screening,health center,2020-2021,82.198953,23
5,no,CDT,screening,health center,2021-2022,68.913561,24
6,no,CDT,screening,health center,2022-2023,40.284360,10
7,no,CDT,screening,health center,2023-2024,40.067340,21
8,no,CDT,treatment,health center,2020-2021,82.091918,23
9,no,CDT,treatment,health center,2021-2022,71.551724,24


In [ ]:
result_df.to_csv("../output/result_df-5.csv", index=False)

PermissionError: [Errno 13] Permission denied: '../output/result_df-5.csv'